In [1]:
import numpy as np
import pandas as pd
import igraph as ig
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from IPython.display import Image

init_notebook_mode(connected=True)

In [2]:
# Helper functions

def dist(A, B):
    return np.linalg.norm(np.array(A)-np.array(B))

def get_idx_interv(d, D):
    k=0
    while(d>D[k]): 
        k+=1
    return  k-1

def deCasteljau(b,t): 
    N=len(b) 
    if(N<2):
        raise InvalidInputError("The  control polygon must have at least two points")
    a=np.copy(b) #shallow copy of the list of control points 
    for r in range(1,N): 
        a[:N-r,:]=(1-t)*a[:N-r,:]+t*a[1:N-r+1,:]                             
    return a[0,:]

def BezierCv(b, nr=5):
    t=np.linspace(0, 1, nr)
    return np.array([deCasteljau(b, t[k]) for k in range(nr)])

Dist=[0, dist([1,0], 2*[np.sqrt(2)/2]), np.sqrt(2), dist([1,0],  [-np.sqrt(2)/2, np.sqrt(2)/2]), 2.0]
params=[1.2, 1.5, 1.8, 2.1]

In [3]:
# Read the data file
df = pd.read_csv('./Data/Action Overlap Bio.csv')
categories = list(set(df['A'].tolist() + df['B'].tolist()))

# Create the graph network
g = ig.Graph.Full(len(categories))
g.vs['name'] = categories
g.es["weight"] = 0.0

for idx, row in df.iterrows():
    if row['A'] == row['B']:
        continue
    # Scale the contribution by a factor of 50000
    g[row['A'], row['B']] += row['n']/50000.0
g.es.select(weight=0.0).delete()

layt = g.layout('circular') #circular layout
Weights = list(map(int, g.es["weight"]))
V=list(g.vs)
labels=[v['name']  for v in V]
E=[e.tuple for e in g.es]# list of edges

node_color=['#FF0000' for v in g.vs] 
line_color=['rgb(342,342,342)' for v in g.vs]
edge_colors=['#000000','#000000', '#000000', '#000000']

Xn=[layt[k][0] for k in range(len(layt))]
Yn=[layt[k][1] for k in range(len(layt))]

lines=[]# the list of dicts defining   edge  Plotly attributes
edge_info=[]# the list of points on edges where  the information is placed

In [4]:
# Percentage contrbutions of every category
marker_size_dict = {}
marker_size_dict['EC'] = 0.149004
marker_size_dict['AP'] = 0.197119
marker_size_dict['RH'] = 0.284524
marker_size_dict['CBC'] = 0.091422
marker_size_dict['RST'] = 0.140321
marker_size_dict['P'] = 0.113154
marker_size_dict['CF'] = 0.024456

# Marker sizes based on Percentage contributions
action_marker_size = []
for label in labels:
    action_marker_size.append(marker_size_dict[label])
for idx in range(len(action_marker_size)):
    action_marker_size[idx] = np.log(action_marker_size[idx]*100)*40

In [5]:
# Plot the Network Analysis
for j, e in enumerate(E):
    A=np.array(layt[e[0]])
    B=np.array(layt[e[1]])
    d=dist(A, B)
    K=get_idx_interv(d, Dist)
    b=[A, A/params[K], B/params[K], B]
    color=edge_colors[K]
    pts=BezierCv(b, nr=5)
    text=V[e[0]]['name']+' to '+V[e[1]]['name']+' '+str(Weights[j])+' x100000'
    mark=deCasteljau(b,0.9)
    
    edge_info.append(go.Scatter(x=[mark[0]],
                             y=[mark[1]],
                             mode='markers',
                             marker=dict(size=0.5, color=edge_colors),
                             text=text,
                             hoverinfo='text'
                             )
                    )
    
    lines.append(go.Scatter(x=pts[:,0],
                         y=pts[:,1],
                         mode='lines',
                         line=dict(color=color,
                                  shape='spline',
                                  width=Weights[j]/250#The  width is proportional to the edge weight
                                 ),
                        text=str(Weights[j]),
                        hoverinfo='text'
                       )
                )

trace2=go.Scatter(x=Xn,
           y=Yn,
           mode='markers+text',
           name='',
           marker=dict(symbol='circle',
                         size=action_marker_size,
                         color=node_color,
                         line=dict(color=line_color, width=0.5)
                         ),
           text=labels,
           hoverinfo='text',
           )

axis=dict(showline=False, # hide axis line, grid, ticklabels and  title
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )


width=800
height=850
layout=go.Layout(title= 'Actions Overlap',
              font= dict(size=20),
              showlegend=False,
              autosize=True,
              width=width,
              height=height,
              xaxis=dict(axis),
              yaxis=dict(axis),
              margin=dict(l=20,
                            r=20,
                            b=85,
                            t=100,
                          ),
              hovermode='closest',
              paper_bgcolor='rgba(0,0,0,0)',
              plot_bgcolor='rgba(0,0,0,0)'
              )

data=lines+edge_info+[trace2]
fig=go.Figure(data=data, layout=layout)

In [6]:
fig.show()